<a href="https://colab.research.google.com/github/jncurrea/Recommender_System/blob/Ibrahim/WorkInProgress_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Crowdsourced Recommender System (WORKING FILE)

### Group Members: Jose Currea, Jenna Ferguson, Evan Hadd, Ramzi Kattan, Hadley Krummel, Jennifer Gonzalez, Ibrahim Muhammad
### Class Section: Afternoon 1 - 3pm

It should accept user inputs in the form of desired attributes of a product and come up with 3 recommendations.

**Your Python Notebook should include the following:**
- All scripts
- The sentiment and similarity scores for the three products you recommended in task E.
- Your analyses for and answer to task F. Make sure you show the ratings, similarity scores and sentiments for the products you recommend in tasks E and F. Use tables whenever possible.  
- Show the logic you are using in addition to finding the most similar product.

## Imports

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from collections import Counter
import re
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
import itertools
from sklearn.manifold import MDS
import statsmodels.api as sm  # For the OLS regression
import numpy as np            # For numerical operations like log transformations
import matplotlib.pyplot as plt  # For plotting
from collections import Counter  # For counting word occurrences
from scipy import stats        # For t-statistic and p-value calculations
from sklearn import manifold

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to multiple lines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


## Task A

Extract about 5-6k reviews. However, many reviews may not have any text and will therefore be discarded. Finally you may end up with 1700-2000 reviews with text.  

In [ ]:
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        messages = soup.find_all("div", class_ = "Message userContent")

        dates = soup.find_all("time")

        data = []

        for message, date in zip(messages, dates):
            message_text = message.get_text(strip = True)
            date_text = date.get("title")
            data.append({"Date": date_text, "Message": message_text})

        return data


def scrape_forum(base_url, total_pages):
    all_data = []

    for page_num in range(1, total_pages + 1):
        page_url = f"{base_url}/p{page_num}"
        print(f"Scraping page {page_num}: {page_url}")
        page_data = scrape_page(page_url)
        all_data.extend(page_data)
    return all_data

In [ ]:
base_url = "https://www.beeradvocate.com/beer/top-rated/"
total_pages = 300
forum_data = scrape_forum(base_url, total_pages)
messagedata = pd.DataFrame(forum_data)
messagedata.to_csv("messagedata.csv", index = False)
len(messagedata)

## Task B

Assume that a customer, who will be using this recommender system, has specified 3 attributes in a product. E.g., one website describes multiple attributes of beer (but you should choose attributes from the actual data like you did for the first assignment)

https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/
- Aggressive (Boldly assertive aroma and/or taste)
- Balanced: Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish
- Complex: Multidimensional; many flavors and sensations on the palate
- Crisp: Highly carbonated; effervescent
- Fruity: Flavors reminiscent of various fruits or Hoppy: Herbal, earthy, spicy, or citric aromas and flavors of hops or Malty: Grainy, caramel-like; can be sweet or dry
- Robust: Rich and full-bodied


In [4]:
#imports & load data
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('beer_comments_final.csv')
df = data

#clean message col; make lowercase, remove special chars + numbers
df['cleaned_comment'] = df['translated_comment'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x.lower()) if isinstance(x, str) else '')
df.head()

,beer_name,overall_rating,translated_comment,cleaned_comment
0,Aecht Schlenkerla Rauchbier Urbock,4.0,probably one of the best smoked ales i have ha...,probably one of the best smoked ales i have ha...
1,Aecht Schlenkerla Rauchbier Urbock,4.0,pint bottle. a medium sized dark brown pour wi...,pint bottle a medium sized dark brown pour wit...
2,Aecht Schlenkerla Rauchbier Urbock,4.0,"clear copper, creamy tan head. aroma of smoked...",clear copper creamy tan head aroma of smoked m...
3,Aecht Schlenkerla Rauchbier Urbock,4.0,dark brown beer with foam. taste is bitterswee...,dark brown beer with foam taste is bittersweet...
4,Aecht Schlenkerla Rauchbier Urbock,4.0,optically clear and very dark chestnut brown. ...,optically clear and very dark chestnut brown t...


In [5]:
#word freq analysis
#countVectorizer to get word freq counts
vectorizer = CountVectorizer(stop_words='english')

#fit + transform cleaned messages to get term-document matrix
X = vectorizer.fit_transform(df['cleaned_comment'])

#sum of counts for each word
word_counts = X.sum(axis=0).tolist()[0]
word_freq = dict(zip(vectorizer.get_feature_names_out(), word_counts))

#top 10
top_words = Counter(word_freq).most_common(10)
print('Top 10 Words by Frequency:')
top_words

Top 10 Words by Frequency:


[('head', 5608),
 ('aroma', 5129),
 ('taste', 3812),
 ('light', 3678),
 ('sweet', 3502),
 ('medium', 3318),
 ('white', 3144),
 ('bottle', 3028),
 ('beer', 2921),
 ('finish', 2721)]

In [6]:
#select top 5 most freq mentioned words as attributes
top_attributes = [word for word, count in Counter(word_freq).most_common(5)]
print('Top 5 Attributes:')
top_attributes #obviously we'll have to look through the real file to get true attributes instead of random top words

Top 5 Attributes:


['head', 'aroma', 'taste', 'light', 'sweet']

In [7]:
#lift analysis
#get cleaned messages + attributes from word freq
input_file = df['cleaned_comment'].apply(lambda x: x.split() if isinstance(x, str) else [])
all_items = top_attributes

#init counters
total_posts = len(input_file)
attribute_counts = {item: 0 for item in all_items}
co_occurrences = {pair: 0 for pair in combinations(all_items, 2)}

#count occurrences + co-occurrences
for tokens in input_file:
    if isinstance(tokens, list):  #checks that tokens are in a list
        #individual occurrences
        unique_tokens = set(tokens)  #avoid double counting in same post
        for token in unique_tokens:
            if token in attribute_counts:
                attribute_counts[token] += 1

        #co-occurrences for attribute pairs
        for attr_a, attr_b in combinations(top_attributes, 2):
            if attr_a in unique_tokens and attr_b in unique_tokens:
                co_occurrences[(attr_a, attr_b)] += 1

#init lift matrix df
lift_matrix = pd.DataFrame(0.0, index=all_items, columns=all_items)

#calc lift for each pair
for pair, co_count in co_occurrences.items():
    attr_a, attr_b = pair
    p_a = attribute_counts[attr_a] / total_posts
    p_b = attribute_counts[attr_b] / total_posts
    p_a_and_b = co_count / total_posts

    if p_a > 0 and p_b > 0: #avoid division by zero
        lift = p_a_and_b / (p_a * p_b)
        if lift_matrix.index.get_loc(attr_a) < lift_matrix.index.get_loc(attr_b):
            lift_matrix.at[attr_a, attr_b] = lift

lift_matrix = lift_matrix.fillna(0.0)

#actual lift matrix
print('Lift Matrix:')
lift_matrix

Lift Matrix:


,head,aroma,taste,light,sweet
head,0.0,1.219022,1.087178,1.167870,1.144358
aroma,0.0,0.000000,1.347719,1.089756,1.205504
taste,0.0,0.000000,0.000000,1.066079,1.125035
light,0.0,0.000000,0.000000,0.000000,1.229145
sweet,0.0,0.000000,0.000000,0.000000,0.000000


In [8]:
#in case you wanted to see the lift values dict
{pair: (co_occurrences[pair] / total_posts) / ((attribute_counts[pair[0]] / total_posts) * (attribute_counts[pair[1]] / total_posts)) for pair in co_occurrences}

{('head', 'aroma'): 1.2190216277969161,
 ('head', 'taste'): 1.0871781083695344,
 ('head', 'light'): 1.1678704049398505,
 ('head', 'sweet'): 1.1443578102327914,
 ('aroma', 'taste'): 1.3477192807831697,
 ('aroma', 'light'): 1.0897555664696406,
 ('aroma', 'sweet'): 1.2055041755205853,
 ('taste', 'light'): 1.0660791887275782,
 ('taste', 'sweet'): 1.1250354298876089,
 ('light', 'sweet'): 1.2291449377281347}

## Task C

Perform a similarity analysis using cosine similarity (without word embeddings – i.e., using the bag-of-words model) with the 3 attributes specified by the customer and the reviews.
The similarity script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name (for each product, the product_name will repeat as many times as there are reviews of the product), product_review and similarity_score.


## Task D

For every review, perform a sentiment analysis (using VADER or any LLM). In case you have to change the default values of words in the VADER lexicon, use this article: https://medium.com/swlh/adding-context-to-unsupervised-sentiment-analysis-7b6693d2c9f8

## Task E

Create an evaluation score for each beer that uses both similarity and sentiment scores.
Now recommend 3 products to the customer.


## Task F

How would your recommendations change if you use word vectors (e.g., the spaCy package with medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity? One way to analyze the difference would be to consider the % of reviews that mention a preferred attribute. E.g., if you recommend a product, what % of its reviews mention an attribute specified by the customer? Do you see any difference across bag-of-words and word vector approaches? Explain. This article may be useful: https://medium.com/swlh/word-embeddings-versus-bag-of-words-the-curious-case-of-recommender-systems-6ac1604d4424?source=friends_link&sk=d746da9f094d1222a35519387afc6338


Note that the article doesn’t claim that bag-of-words will always be better than word embeddings for recommender systems. It lays out conditions under which it is likely to be the case. That is, depending on the attributes you use, you may or may not see the same effect.


In [ ]:
input_file
top_attributes

['head', 'aroma', 'taste', 'light', 'sweet']

In [9]:
# Here we calculate the similarity between the same docs using word vectors (spacy)
!pip install spacy
# download the medium sized file containing word vectors
!python -m spacy download en_core_web_md
import spacy
nlp = spacy.load('en_core_web_md')
text1 = 'How should we end violence in the world?'
text2 = 'What can be done to promote global peace?'
doc1 = nlp(text1)
doc2 = nlp(text2)
print("spaCy similarity:", doc1.similarity(doc2))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
spaCy similarity: 0.757221529115463


In [16]:
# prompt: take the "input_file" data and make a new dataframe that assigns a similarity score from the above step using spacy from "top_attributes". The length of the new dataframe should be equal to "input_file"
# This takes around 2-3 minutes

import pandas as pd
import spacy

# Assuming you have already defined input_file and top_attributes

nlp = spacy.load('en_core_web_md')

similarity_scores = []
for text in input_file:
  text_str = ' '.join(text)  # Convert list of tokens to a string
  doc1 = nlp(text_str)
  doc2 = nlp(' '.join(top_attributes))  # Combine top attributes into a single string
  similarity_scores.append(doc1.similarity(doc2))

new_df = pd.DataFrame({'text': input_file, 'similarity_score': similarity_scores})
print(new_df)



                                                   text  similarity_score
0     [probably, one, of, the, best, smoked, ales, i...          0.719033
1     [pint, bottle, a, medium, sized, dark, brown, ...          0.691132
2     [clear, copper, creamy, tan, head, aroma, of, ...          0.743011
3     [dark, brown, beer, with, foam, taste, is, bit...          0.796948
4     [optically, clear, and, very, dark, chestnut, ...          0.699665
...                                                 ...               ...
9192  [on, tap, at, hill, farm, with, joe, pours, a,...          0.591402
9193  [ml, growler, thanks, to, heather, and, adam, ...          0.726906
9194  [poured, from, growler, thanks, adamchandler, ...          0.819230
9195  [draft, at, big, fatty, s, for, grassroots, ta...          0.682789
9196  [growler, ml, split, ways, bought, earlier, to...          0.705232

[9197 rows x 2 columns]


In [17]:
# prompt: I need to do sentiment analysis of these reviews. I need a score between -1 to 1. There may be sarcasm in the posts in which case you have to correctly identify if this is a negative comment. Use the "translated_comment column in "df" for the sentiment scoring. In "new_df" data frame and add a new column called "sentiment" from this. Bring in the column "beer_name" from "df" as well.

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download VADER lexicon if you haven't already
nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

sentiment_scores = []
for comment in df['translated_comment']:
  if isinstance(comment, str):  # Ensure comment is a string
    scores = analyzer.polarity_scores(comment)
    sentiment_scores.append(scores['compound'])
  else:
    sentiment_scores.append(0)  # Assign a neutral score for non-string comments

new_df['sentiment'] = sentiment_scores
new_df['beer_name'] = df['beer_name']
print(new_df)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                                   text  similarity_score  sentiment                           beer_name
0     [probably, one, of, the, best, smoked, ales, i...          0.719033     0.8402  Aecht Schlenkerla Rauchbier Urbock
1     [pint, bottle, a, medium, sized, dark, brown, ...          0.691132     0.0000  Aecht Schlenkerla Rauchbier Urbock
2     [clear, copper, creamy, tan, head, aroma, of, ...          0.743011     0.3818  Aecht Schlenkerla Rauchbier Urbock
3     [dark, brown, beer, with, foam, taste, is, bit...          0.796948     0.2006  Aecht Schlenkerla Rauchbier Urbock
4     [optically, clear, and, very, dark, chestnut, ...          0.699665     0.3818  Aecht Schlenkerla Rauchbier Urbock
...                                                 ...               ...        ...                                 ...
9192  [on, tap, at, hill, farm, with, joe, pours, a,...          0.591402     0.9488        Hill Farmstead Double Nelson
9193  [ml, growler, thanks, to, 

In [18]:
# prompt: I want to show if the review has the actual attribute mentioned in top_attributes. This should be a binary mention. So i need 5 columns for each of the attribute mentioned in top_attributes, and binary value that shows if it is available in the review.

# Create columns for each attribute with binary values indicating presence
for attribute in top_attributes:
    new_df[attribute] = new_df['text'].apply(lambda x: 1 if attribute in x else 0)

print(new_df)


                                                   text  similarity_score  sentiment                           beer_name  head  aroma  taste  light  sweet
0     [probably, one, of, the, best, smoked, ales, i...          0.719033     0.8402  Aecht Schlenkerla Rauchbier Urbock     0      1      0      0      1
1     [pint, bottle, a, medium, sized, dark, brown, ...          0.691132     0.0000  Aecht Schlenkerla Rauchbier Urbock     1      1      0      0      0
2     [clear, copper, creamy, tan, head, aroma, of, ...          0.743011     0.3818  Aecht Schlenkerla Rauchbier Urbock     1      1      1      0      0
3     [dark, brown, beer, with, foam, taste, is, bit...          0.796948     0.2006  Aecht Schlenkerla Rauchbier Urbock     0      0      1      0      0
4     [optically, clear, and, very, dark, chestnut, ...          0.699665     0.3818  Aecht Schlenkerla Rauchbier Urbock     1      0      0      1      0
...                                                 ...               

In [19]:
# prompt: Make a new column in "new_df" called "final_score" which would be new_df["similarity_score"] * new_df["sentiment"]. Group by "beer_name" by taking average of the final_score. And show me top 3 beers. Also show me what percentage of each attribute is present in these beer brands. Make one table with 3 rows and columns rank, beer_name, score and the attributes. Make an average mentions column too.

import pandas as pd
# Calculate final_score
new_df['final_score'] = new_df['similarity_score'] * new_df['sentiment']

list_groupby = ['final_score', 'similarity_score', 'sentiment' ] + top_attributes

# Group by beer_name and take the average of final_score
#beer_scores =
new_df.groupby('beer_name')[list_groupby].mean().sort_values(by = 'final_score', ascending = False).head(3)


,final_score,similarity_score,sentiment,head,aroma,taste,light,sweet
beer_name,,,,,,,,
The Kernel Lagerbier,0.607081,0.740714,0.820562,0.750000,0.500000,0.62500,0.750000,0.250000
Founders The Headless Gourdsman,0.547588,0.691488,0.792637,0.875000,0.625000,0.50000,0.375000,0.750000
Aecht Schlenkerla Rauchbier Urbock Jahrgangsbier 20xx,0.537146,0.737866,0.725190,0.571429,0.619048,0.47619,0.333333,0.333333


The vector representation form of calculating similarity scores using spacy introduces contextual similarities that Bag of Words method lacks. The traditional BoW method just looks at word frequencies of attributes defined by the user and ranks with that, where as word vectors caputures a deeper semantic meaning for each word by looking at relevant other words in its vector corpus.

## Task G

How would your recommendations differ if you ignored the similarity and feature sentiment scores and simply chose the 3 highest rated products from your entire dataset? Would these products meet the requirements of the user looking for recommendations? Why or why not? Justify your answer with analysis. Use the similarity and sentiment scores as well as overall ratings to answer this question.

In [54]:
# Assuming 'df' contains the original dataframe with 'beer_name' and 'rating' columns

# Group by beer_name and calculate the mean rating
beer_ratings = df.groupby('beer_name')['overall_rating'].mean().sort_values(ascending=False).head(3)

print("Top 3 beers based on average rating:\n")
print(beer_ratings)

# Looking at all the other features of these beers for comparison

filtered_df = new_df[new_df['beer_name'].isin(beer_ratings.index)]
result = filtered_df.groupby('beer_name')[list_groupby].mean()
result = result.merge(beer_ratings, left_index=True, right_index=True, how='left')

result = result.sort_values(by='overall_rating', ascending=False)
result


Top 3 beers based on average rating:

beer_name
Deschutes Fresh Squeezed IPA       4.7
Founders The Headless Gourdsman    4.6
Sante Adairius Cask 200            4.5
Name: overall_rating, dtype: float64


,final_score,similarity_score,sentiment,head,aroma,taste,light,sweet,overall_rating
beer_name,,,,,,,,,
Deschutes Fresh Squeezed IPA,0.325220,0.667762,0.487743,0.468531,0.433566,0.300699,0.146853,0.251748,4.7
Founders The Headless Gourdsman,0.547588,0.691488,0.792637,0.875000,0.625000,0.500000,0.375000,0.750000,4.6
Sante Adairius Cask 200,0.409455,0.722045,0.567111,0.809524,0.531746,0.341270,0.444444,0.119048,4.5


Based on just ratings we see that the rankings for two of the three beer recommendations have changed. Surprisingly "Founders The Headless Gourdsman" is still the 2nd top recommendation for the given attributes.

When we analyze their final score based on our understanding of similarity using spacy and sentiment using Vader, we see that this representation does not hold true under just "ratings" based on the test attributes we chose. Even within the top three, final score is lowest for the highest ranked beer.

Hence a more detailed view that looks into beer similarity with the user attributes defined and the overall sentiment of reviews towards those brands gives a better well rounded recommendation to the end user than just ratings.

## Task H

Choose any 10 beers in your data. Now choose any one of them, and find the most similar beer (among the remaining 9). Explain your method and logic. https://medium.datadriveninvestor.com/who-is-your-competitor-in-the-era-of-the-long-tail-d0ac24fedde8